In [56]:
import os
import time
import lightgbm as lgb
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [58]:
unique_gs_name = 'test_grid_search.csv'

train = pd.read_pickle('data/0.pkl')
val = pd.read_pickle('data/val.pkl')
test = pd.read_pickle('data/test.pkl')

In [3]:
data_path = '/home/kai/data/masaki/titanic/'
train = pd.read_csv(data_path + 'train.csv')

In [4]:
test = pd.read_csv(data_path + 'test.csv')

In [5]:
train.shape

(891, 12)

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
data = pd.concat([train, test])

In [8]:
data.shape

(1309, 12)

In [9]:
categorical_feature = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
print(categorical_feature)
feature_cols = list(set(train.columns) - set(['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin']))
print(feature_cols)
label_col = 'Survived'

['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
['Sex', 'Pclass', 'Parch', 'Fare', 'Age', 'Embarked', 'SibSp']


In [10]:
data[feature_cols].dtypes

Sex          object
Pclass        int64
Parch         int64
Fare        float64
Age         float64
Embarked     object
SibSp         int64
dtype: object

In [11]:
for col in feature_cols:
    num_nulls = np.sum(data[col].isnull())
    if num_nulls != 0:
        col_type = data[col].dtype
        print(col, col_type, num_nulls, num_nulls/len(data[col]))
        if col_type == 'float' or col_type == 'int':
            data[col].fillna(data[col].mean(), inplace=True)
            print('{} filled NA with {}'.format(col, data[col].mean()))
        else:
            data[col].fillna('empty', inplace=True)
            print('{} filled NA with {}'.format(col, 'empty'))

Fare float64 1 0.000763941940413
Fare filled NA with 33.2954792813456
Age float64 263 0.200916730328
Age filled NA with 29.881137667303985
Embarked object 2 0.00152788388083
Embarked filled NA with empty


In [12]:
data.iloc[890:899]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
890,32.0,NaN,Q,7.7500,"Dooley, Mr. Patrick",0,891,3,male,0,0.0,370376
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298
5,14.0,NaN,S,9.2250,"Svensson, Mr. Johan Cervin",0,897,3,male,0,NaN,7538
6,30.0,NaN,Q,7.6292,"Connolly, Miss. Kate",0,898,3,female,0,NaN,330972
7,26.0,NaN,S,29.0000,"Caldwell, Mr. Albert Francis",1,899,2,male,1,NaN,248738


In [13]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
# le_dict = defaultdict(LabelEncoder)
# # Encoding the variable
# fit = df.apply(lambda x: le_dict[x.name].fit_transform(x))

# # Inverse the encoded
# fit.apply(lambda x: le_dict[x.name].inverse_transform(x))

# # Using the dictionary to label future data
# df.apply(lambda x: le_dict[x.name].transform(x))

In [14]:
data[categorical_feature] = data[categorical_feature].apply(LabelEncoder().fit_transform)

In [15]:
train = data[:len(train)]
test = data[len(train):]
train.shape, test.shape

((891, 12), (418, 12))

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_val , y_train, y_val = train_test_split(train[feature_cols], train[label_col], test_size=0.33, random_state=42)

In [18]:
feature_cols

['Sex', 'Pclass', 'Parch', 'Fare', 'Age', 'Embarked', 'SibSp']

In [29]:
import pdb

In [19]:
def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)
    
    return est.strftime(time_format)

In [67]:
def lgb_grid_search(X_train, y_train, X_val, y_val, categorical_feature, search_rounds, filename_for_gs_results, metric='auc', cv=False, nfold=5, verbose_eval=50, X_test=None, preds_save_path=None):
    if X_test != None:
        if preds_save_path == None:
            print('No save path provides, "SUBS" will be created at the current directory')
            os.makedirs('SUBS/')
        else:
            if not os.path.exists(preds_save_path):
                raise ValueError('{} path does not exist. Mission aborted.'.format())
                    
    for i in range(search_rounds):
#         try:
        seed = int(time.time()* 1000000) % 45234634
        np.random.seed(seed)
        run_id = int(time.time()) # also works as the index of the result dataframe

        lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_feature)#train[feature_cols], train[label_col], categorical_feature=categorical_feature)

        lgb_val = lgb.Dataset(X_val, y_val, categorical_feature=categorical_feature)#val[feature_cols], val[label_col], categorical_feature=categorical_feature)

        lgb_params = {
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_rounds': 4000,
            'learning_rate': np.random.choice([0.1,0.03,0.001]),
            'num_leaves': np.random.choice([15,31,61,127]),
            'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
#                 'device': 'cpu',
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        #     'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
            'feature_fraction': np.random.randint(3,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':70,
            'bagging_fraction': np.random.randint(3,11)/10, #Randomly select part of data 
            'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
            'bagging_seed': seed,
            'scale_pos_weight': 2,
            'metric' : metric
        }
#             pp.pprint(lgb_params)

        lgb_params['timestamp'] = get_time()
        if cv:
            eval_hist = lgb.cv(lgb_params, lgb_train, nfold=nfold, 
                               categorical_feature=categorical_feature, verbose_eval=verbose_eval, seed=seed)
            lgb_params['best_round'] = len(eval_hist[metric+'-mean']) 
            lgb_params['val_'+metric] = eval_hist[metric+'-mean'][-1]
            lgb_params['cv'] = True
            if X_test != None:
                model = lgb.train(lgb_params, lgb_train, valid_sets=lgb_train, verbose_eval=verbose_eval)
                y_test = model.predict(X_test)
                
        else:
            model = lgb.train(lgb_params, lgb_train, valid_sets=[lgb_train, lgb_val], verbose_eval=verbose_eval)
            lgb_params['best_round'] = model.best_iteration
            lgb_params['val_'+metric] = model.best_score['valid_1'][metric]
            lgb_params['train_'+metric] = model.best_score['training'][metric]
        

        for k, v in lgb_params.items():
            if isinstance(v, list):
                lgb_params[k] = '"'+str(v)+'"'
                print(lgb_params[k])

        res = pd.DataFrame(lgb_params, index=[run_id])

        if not os.path.exists(filename_for_gs_results):
            res.to_csv(filename_for_gs_results)
            print(filename_for_gs_results, 'created')
        else:
            old_res = pd.read_csv(filename_for_gs_results, index_col='Unnamed: 0')
            res = pd.concat([old_res, res])
            res.to_csv(filename_for_gs_results)
            print(filename_for_gs_results, 'updated')
                
#         except Exception as e:
#             if 'ResourceExhaustedError' in str(type(e)): # can't catch this error directly... 
#                 print('Oops! ResourceExhaustedError. Continue next round')
#                 continue
#             else:
#                 print(e)
#                 break

In [68]:
lgb_grid_search(X_train, y_train, X_val, y_val, 
                categorical_feature=categorical_feature, search_rounds=2, 
                filename_for_gs_results=unique_gs_name, cv=True)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:395: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[50]	cv_agg's auc: 0.822771 + 0.0307683
"[0, 1, 2, 5, 6]"
test_grid_search.csv updated
[50]	cv_agg's auc: 0.836995 + 0.0137805
[100]	cv_agg's auc: 0.84365 + 0.0140243
[150]	cv_agg's auc: 0.851525 + 0.0176166
[200]	cv_agg's auc: 0.854532 + 0.0154343
[250]	cv_agg's auc: 0.85781 + 0.0161793
[300]	cv_agg's auc: 0.85781 + 0.0181022
[350]	cv_agg's auc: 0.860841 + 0.0199124
[400]	cv_agg's auc: 0.861605 + 0.0180578
[450]	cv_agg's auc: 0.863159 + 0.0163448
[500]	cv_agg's auc: 0.865324 + 0.016476
[550]	cv_agg's auc: 0.868924 + 0.0151539
[600]	cv_agg's auc: 0.867307 + 0.01682
"[0, 1, 2, 5, 6]"
test_grid_search.csv updated


In [69]:
t1 = pd.read_csv(unique_gs_name, index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)

In [70]:
t1

,bagging_fraction,bagging_freq,bagging_seed,best_round,boosting,categorical_column,cv_val_auc,device,feature_fraction,feature_fraction_seed,...,max_depth,metric,num_leaves,num_threads,objective,scale_pos_weight,timestamp,train_auc,val_auc,verbose
1528746580,0.9,1,10686593,26,gbdt,"""[0, 1, 2, 5, 6]""",NaN,NaN,0.9,10686593,...,-1,auc,31,4,binary,2.0,2018-06-11 15:49:40,0.938575,0.887643,1
1528737157,0.7,1,41646154,25,gbdt,"""[0, 1, 2, 3, 6]""",NaN,NaN,0.8,41646154,...,-1,"""['auc']""",31,4,binary,2.0,2018-06-11 13:12:37,0.919702,0.886262,1
1528668075,0.7,1,1,160,gbdt,"""[0, 1, 2, 3, 6]""",NaN,cpu,0.7,1,...,-1,"""['auc']""",127,4,binary,NaN,NaN,0.933468,0.885929,0
1528667347,0.7,1,1,158,gbdt,"""[0, 1, 2, 3, 6]""",NaN,cpu,0.7,1,...,-1,"""['auc']""",15,4,binary,NaN,NaN,0.931854,0.885881,0
1528746580,0.7,1,10774865,139,gbdt,"""[0, 1, 2, 5, 6]""",NaN,NaN,0.8,10774865,...,-1,auc,61,4,binary,2.0,2018-06-11 15:49:40,0.892916,0.883619,1
1528737157,0.3,1,41827884,64,gbdt,"""[0, 1, 2, 3, 6]""",NaN,NaN,0.7,41827884,...,-1,"""['auc']""",127,4,binary,2.0,2018-06-11 13:12:37,0.897535,0.883190,1
1528734235,0.7,1,1,36,gbdt,"""[0, 1, 2, 3, 6]""",NaN,cpu,0.7,1,...,-1,"""['auc']""",127,4,binary,2.0,2018-06-11 12:23:55,0.927247,0.882119,0
1528734457,0.7,1,1,36,gbdt,"""[0, 1, 2, 3, 6]""",NaN,cpu,0.7,1,...,-1,"""['auc']""",31,4,binary,2.0,2018-06-11 12:27:37,0.927247,0.882119,1
1528734457,0.7,1,1,36,gbdt,"""[0, 1, 2, 3, 6]""",NaN,cpu,0.7,1,...,-1,"""['auc']""",31,4,binary,2.0,2018-06-11 12:27:38,0.927247,0.882119,1
1528668143,0.7,1,1,161,gbdt,"""[0, 1, 2, 3, 6]""",NaN,cpu,0.7,1,...,-1,"""['auc']""",15,4,binary,2.0,NaN,0.887454,0.876667,0


In [101]:
temp = pd.read_csv(unique_gs_name)

In [57]:
results_header

'objective,boosting,learning_rate,num_leaves,num_threads,device,max_depth,feature_fraction,feature_fraction_seed,bagging_fraction,bagging_freq,bagging_seed,verbose,metric,categorical_column,best_round,val_auc,train_auc\n'

In [60]:
list(params_wrapper.keys())

['objective',
 'boosting',
 'learning_rate',
 'num_leaves',
 'num_threads',
 'device',
 'max_depth',
 'feature_fraction',
 'feature_fraction_seed',
 'bagging_fraction',
 'bagging_freq',
 'bagging_seed',
 'verbose',
 'metric',
 'categorical_column',
 'best_round',
 'val_auc',
 'train_auc']

In [64]:
np.sum(res.columns.values != list(params_wrapper.keys()))

0

In [50]:
old_head = 

In [51]:
old_head

<bound method NDFrame.head of   objective boosting  learning_rate  num_leaves  num_threads device  \
0    binary     gbdt            0.1          31            4    cpu   

   max_depth  feature_fraction  feature_fraction_seed  bagging_fraction  \
0         -1               0.7                      1               0.7   

   bagging_freq  bagging_seed  verbose   metric categorical_column  \
0             1             1        0  ['auc']    [0, 1, 2, 3, 6]   

   best_round   val_auc  train_auc  
0          45  0.884071   0.928867  >

In [49]:
res.head()

,objective,boosting,learning_rate,num_leaves,num_threads,device,max_depth,feature_fraction,feature_fraction_seed,bagging_fraction,bagging_freq,bagging_seed,verbose,metric,categorical_column,best_round,val_auc,train_auc
0,binary,gbdt,0.1,31,4,cpu,-1,0.7,1,0.7,1,1,0,['auc'],"[0, 1, 2, 3, 6]",45,0.884071,0.928867


In [114]:
X = pd.DataFrame({"A": np.random.permutation(['a', 'b', 'c', 'd'] * 75),  # str 
                       "B": np.random.permutation([1, 2, 3] * 100),  # int 
                       "C": np.random.permutation([0.1, 0.2, -0.1, -0.1, 0.2] * 60),  # float 
                       "D": np.random.permutation([True, False] * 150)})  # bool 

In [53]:
X.head()

NameError: name 'X' is not defined